# **Fase inicial: Preprocesamiento de los datos**
---
### Fusión todos los archivos con extension .pcap del conjunto de datos CTU-13:  

Con el fin de realizar diversos análisis, se optó por transformar los archivos del conjunto CTU-13 a formato de dataframe, posteriormente, además de fusionarlos en un solo archivo con formato dataframe de la librería pandas.

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
data_path = './CTU-13_data' #Ruta de la carpeta con archivos .pcap correspondientes

data_file_names = [file for file in os.listdir(data_path)] # Lectura de los nombres de los archivos del directorio
#print(data_file_names)  # Lista de strings que contiene el nombre de los archivos .parquet

### Iteramos sobre los archivos que estan en la ruta especificada en ``` data_path = './CTU-13_data' ``` donde se hace la lectura de los archivos de .pcap para convertirlos a .csv.

In [2]:
# Diccionario que contendra los dataframes despues de hacer el cast de parquet a pandas df
data_frames = {}
'''
Iteramos sobre la lista de nombres para añadirlos al diccionario de dataframes

Ejemplo del formato del diccionario seria: 

    {'1-Neris-20110810.binetflow.parquet': pd.DataFrame('./CTU-13_data/1-Neris-20110810.binetflow.parquet')}

'''
for file in data_file_names:
    data_frames[file] = pd.read_parquet(f'{data_path}/{file}') 

### Ahora fusionamos todos los datasets

In [3]:
all_data = pd.concat([ data_frames[file] for file in data_file_names ])
print(all_data.head())
print(all_data.shape)
print(all_data.info()) #Visualizar la informacion y tipo de datos para cada columna

        dur proto    dir state  stos  dtos  tot_pkts  tot_bytes  src_bytes  \
0  1.026539   tcp     ->  S_RA   0.0   0.0         4        276        156   
1  1.009595   tcp     ->  S_RA   0.0   0.0         4        276        156   
2  3.056586   tcp     ->  SR_A   0.0   0.0         3        182        122   
3  3.111769   tcp     ->  SR_A   0.0   0.0         3        182        122   
4  3.083411   tcp     ->  SR_A   0.0   0.0         3        182        122   

                                    label                  Family  
0  flow=Background-Established-cmpgw-CVUT  20110810.binetflow.csv  
1  flow=Background-Established-cmpgw-CVUT  20110810.binetflow.csv  
2             flow=Background-TCP-Attempt  20110810.binetflow.csv  
3             flow=Background-TCP-Attempt  20110810.binetflow.csv  
4             flow=Background-TCP-Attempt  20110810.binetflow.csv  
(10598771, 11)
<class 'pandas.core.frame.DataFrame'>
Index: 10598771 entries, 0 to 1274631
Data columns (total 11 columns):

In [4]:
#Visualizar una muestra del contenido de los registros:
all_data.sample(10)


,dur,proto,dir,state,stos,dtos,tot_pkts,tot_bytes,src_bytes,label,Family
8557,2904.621826,udp,<->,CON,0.0,0.0,4,266,146,flow=Background-UDP-Established,20110815
86689,0.074338,udp,<->,CON,0.0,0.0,2,148,74,flow=To-Background-UDP-CVUT-DNS-Server,20110812.binetflow.csv
1243375,307.979309,tcp,->,FSPA_FSPA,0.0,0.0,58,42259,1793,flow=Background-Established-cmpgw-CVUT,20110817.binetflow.csv
869869,0.000315,udp,<->,CON,0.0,0.0,2,259,92,flow=To-Background-UDP-CVUT-DNS-Server,20110810.binetflow.csv
312898,15.461116,tcp,->,FSPA_FSPA,0.0,0.0,16,5612,1232,flow=Background-TCP-Established,20110810.binetflow.csv
1144512,0.072780,tcp,->,FSPA_FSPA,0.0,0.0,17,7459,1103,flow=Background-TCP-Established,20110817.binetflow.csv
985090,2451.488770,udp,<->,CON,0.0,0.0,4,264,144,flow=Background-UDP-Established,20110817.binetflow.csv
138297,2091.429443,udp,<->,CON,0.0,0.0,4,272,152,flow=Background-UDP-Established,20110810.binetflow.csv
88586,5.063198,tcp,->,SRPA_FSPA,0.0,0.0,10,2146,1323,flow=Background-Established-cmpgw-CVUT,20110818.binetflow.csv
818886,4.757684,udp,->,INT,0.0,NaN,3,1428,1428,flow=Background-UDP-Attempt,20110811.binetflow.csv


Las columnas "label" y "family" serán modificadas para su personalización.

1. Análisis de la columna *label*:  
- Se observa que la columna posee una etiqueta de flow, donde indica el tipo de conexipon solicitada/establecida, o bien, el registro de la presencia de un Botnet.  
- Se propone el método de etiquetado 0 == normal, 1 == ataque (presencia de botnets)


In [5]:
#Visualizacion de las columnas con registro de Botnet:
print("Registros que corresponden a ataques:")
print(all_data[all_data['label'].str.contains('Botnet')].shape[0])
print("Registros que corresponden a tráfico normal:")
print(all_data[all_data['label'].str.contains('Botnet') == False].shape[0])

Registros que corresponden a ataques:
262573
Registros que corresponden a tráfico normal:
10336198


In [6]:
dfmod = all_data #Crear una copia del dataframe original para su modificacion

# Crear una nueva columna evaluando si "botnet" está en la columna "label"
dfmod["is_botnet"] = dfmod["label"].astype(str).apply(lambda x: 1 if "botnet" in x.lower() else 0)

dfmod.sample(18, random_state=432)

,dur,proto,dir,state,stos,dtos,tot_pkts,tot_bytes,src_bytes,label,Family,is_botnet
425570,0.001110,udp,<->,CON,0.0,0.0,2,618,75,flow=Background-UDP-Established,20110810.binetflow.csv,0
32523,0.000655,udp,->,INT,0.0,NaN,1,107,107,flow=Background-UDP-Attempt,20110815.binetflow.csv,0
1187347,264.363129,tcp,->,SRPA_SPA,0.0,0.0,54,6301,3301,flow=Background-TCP-Established,20110812.binetflow.csv,0
658905,0.135019,udp,<->,CON,0.0,0.0,2,313,64,flow=Background-UDP-Established,20110818.binetflow.csv,0
689170,0.000452,udp,<->,CON,0.0,0.0,2,234,86,flow=To-Background-UDP-CVUT-DNS-Server,20110815.binetflow.csv,0
71394,0.029950,udp,<->,CON,0.0,0.0,2,158,71,flow=Background-UDP-Established,20110818.binetflow.csv,0
1092222,0.135506,tcp,->,FSPA_FSPA,0.0,0.0,33,25654,853,flow=Background-TCP-Established,20110816,0
1256117,1531.330200,udp,<->,CON,0.0,0.0,10,1858,344,flow=Background-UDP-Established,20110810.binetflow.csv,0
1015277,9.032158,tcp,->,S_,0.0,NaN,3,186,186,flow=From-Botnet-V43-TCP-Attempt-SPAM,20110811.binetflow.csv,1
635089,0.004251,tcp,->,FSPA_FSPA,0.0,0.0,10,1343,810,flow=Background-TCP-Established,20110815.binetflow.csv,0
